In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv('salario.csv')
print(df)

       id           pais         estado              cidade  genero  \
0       1  United States        Alabama              Mobile    Male   
1       2       Portugal          Porto                Rego    Male   
2       3         Sweden         Kalmar            Emmaboda    Male   
3       4         Sweden    Västmanland                Sala  Female   
4       5         France       Bretagne           Quimperlé  Female   
..    ...            ...            ...                 ...     ...   
995   996         Canada   Saskatchewan             Nipawin  Female   
996   997       Portugal     Portalegre        Santo Aleixo  Female   
997   998         France  Île-de-France  Boissy-Saint-Léger    Male   
998   999         France       Bretagne              Rennes    Male   
999  1000         Mexico     Guanajuato          Lindavista  Female   

           data   salario  
0    10/01/1987  $5407.86  
1    29/11/2007  $7833.82  
2    23/06/1984  $5530.85  
3    04/11/1983  $2803.76  
4    18